## Call Notebook to Setup Environment and Initial Imports

In [3]:
%run "Setup_Env.ipynb"

## Mini-Project 1: Review Analyst

You are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

  - Summarize the review. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response

Try to get the response in a nice structured format using an output parser

### Why Both Partial Variable AND Parser Are Needed

**The Flow:**
1. **Partial Variable** (`format_instructions`) → Gets injected into prompt → **Tells LLM how to respond**
2. **LLM** → Generates text response following those instructions
3. **Parser** → Converts that text response → **Into Python object**

**Think of it like this:**
- `format_instructions` = \"Please fill out this form in JSON format\"  
- `parser` = Takes the completed form and creates a structured object from it

**Without partial variable:** LLM wouldn't know how to structure its response  
**Without parser:** You'd get unstructured text instead of a clean Python object


In [4]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

### Define Output Parser

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define your desired data structure - like a python data class.
class Review(BaseModel):
    summary: str = Field(description="summary of the review")
    positives: list[str] = Field(description="list of positive aspects of the review if any - max 3 points")
    negatives: list[str] = Field(description="list of negative aspects of the review if any - max 3 points")
    sentiment: str = Field(description="overall sentiment of the review - positive, negative or neutral")
    emotions: list[str] = Field(description="list of emotions expressed by the customer in the review - max 5 points")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Review)
format_instructions = parser.get_format_instructions()



In [10]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Analyze the given customer review below and generate the response based on the instructions
             mentioned below in the format instructions.
             Also remember to write a detailed email response for the email field based on these conditions:
               - email should be addressed to Dear Customer and signed with Service Agent
               - thank them if the review is positive or neutral
               - apologize if the review is negative

             Format Instructions:
             {format_instructions}

             Review:
             {review}
            """

prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["review"],
    partial_variables={"format_instructions": format_instructions}
)



In [11]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
           |
         chatgpt
           |
         parser)

In [14]:
print(reviews[0])


    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    


In [18]:
sample_response = chain.invoke({"review": reviews[0]})
print(sample_response.summary)

The customer is very satisfied with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money.


In [19]:
sample_response.model_dump_json()

'{"summary":"The customer is very satisfied with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money.","positives":["Impressive sound quality with clear audio and good bass","Waterproof feature works effectively","Compact size with powerful volume"],"negatives":[],"sentiment":"positive","emotions":["satisfaction","happiness","excitement"]}'

In [20]:
reviews_formatted = [{'review': review} for review in reviews]
print(reviews_formatted[0])

{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "}


In [21]:
responses = chain.map().invoke(reviews_formatted)

In [22]:
responses[0].model_dump()

{'summary': 'The customer is very satisfied with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money.',
 'positives': ['Impressive sound quality with clear audio and good bass',
  'Waterproof feature works effectively',
  'Compact size with powerful volume'],
 'negatives': [],
 'sentiment': 'positive',
 'emotions': ['satisfaction', 'happiness', 'excitement']}

In [23]:
responses[0].model_dump_json()

'{"summary":"The customer is very satisfied with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money.","positives":["Impressive sound quality with clear audio and good bass","Waterproof feature works effectively","Compact size with powerful volume"],"negatives":[],"sentiment":"positive","emotions":["satisfaction","happiness","excitement"]}'

In [24]:
responses[1].model_dump_json()

'{"summary":"The customer is very satisfied with their new gaming keyboard, highlighting its responsiveness, backlighting, and overall value.","positives":["Satisfying click of the keys","Vibrant LED colors enhancing gaming experience","Competitive pricing and good deal"],"negatives":[],"sentiment":"positive","emotions":["satisfaction","excitement","contentment"]}'

In [25]:
responses[2].model_dump_json()

'{"summary":"The customer is disappointed with the wireless earbuds due to sound issues, comfort, and battery life.","positives":["The earbuds arrived on time."],"negatives":["Sound constantly cuts out.","Uncomfortable fit after a few minutes.","Battery life is only four hours instead of the advertised 12 hours."],"sentiment":"negative","emotions":["disappointment","frustration","dissatisfaction"]}'

In [26]:
for response in responses:
  for k,v in response.dict().items():
    print(f'{k}:\n{v}')
  print('-----'*50)
  print('\n')

summary:
The customer is very satisfied with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money.
positives:
['Impressive sound quality with clear audio and good bass', 'Waterproof feature works effectively', 'Compact size with powerful volume']
negatives:
[]
sentiment:
positive
emotions:
['satisfaction', 'happiness', 'excitement']
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


summary:
The customer is very satisfied with their new gaming keyboard, highlighting its responsiveness, backlighting, and overall value.
positives:
['Satisfying click of the keys', 'Vibrant LED colors enhancing gaming experience', 'Competitive pricing and good deal']
negatives:
[]
sentiment:
positive
emotions:
['satisfaction', 'excitement', 'contentment']
-----------

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_51098/2504484312.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for k,v in response.dict().items():


In [27]:
for response in responses:
    print(response)

summary='The customer is very satisfied with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money.' positives=['Impressive sound quality with clear audio and good bass', 'Waterproof feature works effectively', 'Compact size with powerful volume'] negatives=[] sentiment='positive' emotions=['satisfaction', 'happiness', 'excitement']
summary='The customer is very satisfied with their new gaming keyboard, highlighting its responsiveness, backlighting, and overall value.' positives=['Satisfying click of the keys', 'Vibrant LED colors enhancing gaming experience', 'Competitive pricing and good deal'] negatives=[] sentiment='positive' emotions=['satisfaction', 'excitement', 'contentment']
summary='The customer is disappointed with the wireless earbuds due to sound issues, comfort, and battery life.' positives=['The earbuds arrived on time.'] negatives=['Sound constantly cuts out.', 'Uncomfortable fit after a few minutes.', 'Battery life is only four 

In [28]:
import pandas as pd

pd.DataFrame(response.dict() for response in responses)

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_51098/1175396765.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pd.DataFrame(response.dict() for response in responses)


,summary,positives,negatives,sentiment,emotions
0,The customer is very satisfied with the Blueto...,[Impressive sound quality with clear audio and...,[],positive,"[satisfaction, happiness, excitement]"
1,The customer is very satisfied with their new ...,"[Satisfying click of the keys, Vibrant LED col...",[],positive,"[satisfaction, excitement, contentment]"
2,The customer is disappointed with the wireless...,[The earbuds arrived on time.],"[Sound constantly cuts out., Uncomfortable fit...",negative,"[disappointment, frustration, dissatisfaction]"
3,The customer is disappointed with the tablet s...,[The product arrived promptly.],"[The stand wobbles with the slightest touch., ...",negative,"[disappointment, frustration]"
4,The customer is dissatisfied with the kitchen ...,[],"[Struggles with tougher foods, Incredibly nois...",negative,"[frustration, disappointment, anger]"


In [29]:
# Convert the list of responses to a pandas DataFrame
df = pd.DataFrame([response.model_dump() for response in responses])
df


,summary,positives,negatives,sentiment,emotions
0,The customer is very satisfied with the Blueto...,[Impressive sound quality with clear audio and...,[],positive,"[satisfaction, happiness, excitement]"
1,The customer is very satisfied with their new ...,"[Satisfying click of the keys, Vibrant LED col...",[],positive,"[satisfaction, excitement, contentment]"
2,The customer is disappointed with the wireless...,[The earbuds arrived on time.],"[Sound constantly cuts out., Uncomfortable fit...",negative,"[disappointment, frustration, dissatisfaction]"
3,The customer is disappointed with the tablet s...,[The product arrived promptly.],"[The stand wobbles with the slightest touch., ...",negative,"[disappointment, frustration]"
4,The customer is dissatisfied with the kitchen ...,[],"[Struggles with tougher foods, Incredibly nois...",negative,"[frustration, disappointment, anger]"
